In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv("data/raw/loan_data.csv")

In [3]:
df.head()

ApplicationDate  Age  AnnualIncome  CreditScore EmploymentStatus  \
0      2018-01-01   45         97587          485         Employed   
1      2018-01-02   38        112562          524         Employed   
2      2018-01-03   47         37175          557         Employed   
3      2018-01-04   58         96585          566         Employed   
4      2018-01-05   37         18018          539         Employed   

  EducationLevel  Experience  LoanAmount  LoanDuration MaritalStatus  ...  \
0    High School          26       31180            48       Married  ...   
1         Master          18       10753            96        Single  ...   
2      Doctorate          26       18787            24       Married  ...   
3       Bachelor          35       27023            24       Married  ...   
4    High School          12       21464            60       Married  ...   

   MonthlyIncome UtilityBillsPaymentHistory  JobTenure  NetWorth  \
0    8132.250000                   0.728416          7      3456   
1    9380.166667                   0.817123          3      6250   
2    3097.916667                   0.923235          4     72140   
3    8048.750000                   0.827937          5      9565   
4    1501.500000                   0.640931          3     15878   

   BaseInterestRate  InterestRate  MonthlyLoanPayment  TotalDebtToIncomeRatio  \
0          0.283680      0.298853         1120.587305                0.148863   
1          0.283753      0.253999          262.787193                0.055307   
2          0.215287      0.242619          995.746474                0.417295   
3          0.219023      0.241088         1430.204114                0.217078   
4          0.256964      0.254586          635.780171                0.722464   

  LoanApproved  RiskScore  
0            0       53.0  
1            1       43.2  
2            0       56.0  
3            1       40.8  
4            0       55.0  

[5 rows x 36 columns]

In [4]:
df.shape

(30000, 36)

# Limpando os dados

In [5]:
def limpar_dados(df):
    df = df.drop(columns=['ApplicationDate', 'LoanPurpose', 'MaritalStatus', 'LoanApproved'])
    df = df.dropna()
    df['EducationLevel'] = df['EducationLevel'].astype('category').cat.codes
    df['EmploymentStatus'] = df['EmploymentStatus'].astype('category').cat.codes
    df['HomeOwnershipStatus'] = df['HomeOwnershipStatus'].astype('category').cat.codes
    return df

In [6]:
df_tratado = limpar_dados(df)

# Splitar dados

In [7]:
X = df_tratado.drop(columns=['RiskScore'])
y = df_tratado['RiskScore']

In [8]:
X

Age  AnnualIncome  CreditScore  EmploymentStatus  EducationLevel  \
0       45         97587          485                 0               3   
1       38        112562          524                 0               4   
2       47         37175          557                 0               2   
3       58         96585          566                 0               1   
4       37         18018          539                 0               3   
...    ...           ...          ...               ...             ...   
29995   33         80018          474                 0               3   
29996   57         26050          587                 0               3   
29997   33         30868          520                 0               0   
29998   18         33660          571                 0               3   
29999   37         71037          515                 0               3   

       Experience  LoanAmount  LoanDuration  NumberOfDependents  \
0              26       31180            48                   1   
1              18       10753            96                   4   
2              26       18787            24                   2   
3              35       27023            24                   2   
4              12       21464            60                   1   
...           ...         ...           ...                 ...   
29995          12       75495            36                   3   
29996          37       10306           120                   1   
29997          10       12658            24                   2   
29998           0       13668            72                   0   
29999          15       10854            72                   1   

       HomeOwnershipStatus  ...  TotalAssets  TotalLiabilities  MonthlyIncome  \
0                        2  ...        30103             35450    8132.250000   
1                        2  ...         7927             35333    9380.166667   
2                        0  ...       130671             58531    3097.916667   
3                        0  ...        27318             17753    8048.750000   
4                        3  ...        65989             50111    1501.500000   
...                    ...  ...          ...               ...            ...   
29995                    3  ...        23154              5096    6668.166667   
29996                    0  ...        25168             28154    2170.833333   
29997                    2  ...        47649             22246    2572.333333   
29998                    2  ...        48153             17877    2805.000000   
29999                    2  ...        36412             10184    5919.750000   

       UtilityBillsPaymentHistory  JobTenure  NetWorth  BaseInterestRate  \
0                        0.728416          7      3456          0.283680   
1                        0.817123          3      6250          0.283753   
2                        0.923235          4     72140          0.215287   
3                        0.827937          5      9565          0.219023   
4                        0.640931          3     15878          0.256964   
...                           ...        ...       ...               ...   
29995                    0.609454          4     18058          0.323495   
29996                    0.665533          5      2477          0.271806   
29997                    0.770767          5     25403          0.227658   
29998                    0.625701          3     30276          0.243168   
29999                    0.938527          6     26228          0.268354   

       InterestRate  MonthlyLoanPayment  TotalDebtToIncomeRatio  
0          0.298853         1120.587305                0.148863  
1          0.253999          262.787193                0.055307  
2          0.242619          995.746474                0.417295  
3          0.241088         1430.204114                0.217078  
4          0.254586          635.780171                0.722464  
...             ...

In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [10]:
X_scaled

array([[ 0.4592164 ,  0.97506009, -1.67199118, ...,  1.41316062,
         0.31252225, -0.73893539],
       [-0.14430857,  1.35237425, -0.91515485, ...,  0.35652728,
        -0.980861  , -1.00984411],
       [ 0.6316521 , -0.54709702, -0.27475488, ...,  0.08843459,
         0.12428837,  0.03836421],
       ...,
       [-0.57539783, -0.7060099 , -0.99277909, ...,  0.80860483,
        -0.33612017,  0.14414747],
       [-1.86866561, -0.63566191, -0.00307004, ...,  0.53933031,
        -0.80699241,  0.1184598 ],
       [-0.23052642,  0.3060991 , -1.08980939, ..., -0.09598075,
        -0.95139667, -0.74377824]])

y

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

# Testing Linear Regression

In [12]:
linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train, y_train)

y_pred = linear_regression_model.predict(X_test)

In [13]:
mse = mean_squared_error(y_test, y_pred, squared=True)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse}\nRMSE: {rmse}\nMAE: {mae}\nMAPE: {mape}\nR2 Score: {r2}')

MSE: 13.574909025090122
RMSE: 3.684414339496865
MAE: 2.90516737693993
MAPE: 0.0620646787270151
R2 Score: 0.7783156234272155


/opt/homebrew/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# K-NN regressor

In [14]:
knn_model = KNeighborsRegressor(n_neighbors=7, metric='euclidean')
knn_model.fit(X_train, y_train)
y_pred = knn_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred, squared=True)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse}\nRMSE: {rmse}\nMAE: {mae}\nMAPE: {mape}\nR2 Score: {r2}')

MSE: 16.344565333333332
RMSE: 4.042841245131118
MAE: 3.0692114285714283
MAPE: 0.06441846402504332
R2 Score: 0.7330858888574328


/opt/homebrew/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Support Vector Machines for Regressor (SVR)

In [15]:
#model = SVR(kernel='linear', C=100)
#model.fit(X_train, y_train)
#y_pred = model.predict(X_test)

#mse = mean_squared_error(y_test, y_pred, squared=True)
#rmse = np.sqrt(mse)
#mae = mean_absolute_error(y_test, y_pred)
#mape = mean_absolute_percentage_error(y_test, y_pred)
#print(f'MSE: {mse}\nRMSE: {rmse}\nMAE: {mae}\nMAPE: {mape}')

# Árvore de regressão

In [16]:
decision_tree_model = DecisionTreeRegressor()
decision_tree_model.fit(X_train, y_train)
y_pred = decision_tree_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred, squared=True)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse}\nRMSE: {rmse}\nMAE: {mae}\nMAPE: {mape}\nR2 Score: {r2}')

MSE: 16.326410666666664
RMSE: 4.040595335673527
MAE: 2.1322666666666668
MAPE: 0.04536533431090006
R2 Score: 0.7333823627383588


/opt/homebrew/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# XGBoost

In [17]:
xgboost_model = XGBRegressor()
xgboost_model.fit(X_train, y_train)
y_pred = xgboost_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred, squared=True)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse}\nRMSE: {rmse}\nMAE: {mae}\nMAPE: {mape}\nR2 Score: {r2}')

MSE: 5.775048255997675
RMSE: 2.403133008386692
MAE: 1.5572231912740069
MAPE: 0.03332116669799235
R2 Score: 0.9056908617256761


/opt/homebrew/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Random Forest Regressor

In [18]:
random_forest_model = RandomForestRegressor()
random_forest_model.fit(X_train, y_train)
y_pred = random_forest_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred, squared=True)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse}\nRMSE: {rmse}\nMAE: {mae}\nMAPE: {mape}\nR2 Score: {r2}')

MSE: 6.842863891199998
RMSE: 2.6158868269097573
MAE: 1.5809477333333333
MAPE: 0.03380837187215711
R2 Score: 0.8882529516117315


/opt/homebrew/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


# Testar com outros dados

In [19]:
df_test = pd.read_csv("data/raw/loan_data_test.csv")

In [20]:
df_test = limpar_dados(df_test)

In [21]:
df_test

Age  AnnualIncome  CreditScore  EmploymentStatus  EducationLevel  \
0       36         17021          581                 0               0   
1       41         49421          559                 0               4   
2       43         30529          587                 0               3   
3       28         50218          592                 0               4   
4       46         79758          595                 0               3   
...    ...           ...          ...               ...             ...   
29995   26         65143          588                 0               1   
29996   19         55780          555                 0               0   
29997   18         37733          549                 0               1   
29998   18         27579          479                 0               4   
29999   38         98314          569                 0               0   

       Experience  LoanAmount  LoanDuration  NumberOfDependents  \
0              12        7332            60                   4   
1              16       15713            60                   3   
2              21       63870            48                   0   
3               8       31051            24                   2   
4              25       16971            48                   0   
...           ...         ...           ...                 ...   
29995           5       69762            48                   0   
29996           0       28513            84                   3   
29997           0       18899            36                   3   
29998           0        7787           108                   0   
29999          12       25969            24                   1   

       HomeOwnershipStatus  ...  TotalLiabilities  MonthlyIncome  \
0                        0  ...              5848    1418.416667   
1                        0  ...             21838    4118.416667   
2                        3  ...            105717    2544.083333   
3                        2  ...             51145    4184.833333   
4                        2  ...             11461    6646.500000   
...                    ...  ...               ...            ...   
29995                    2  ...             31537    5428.583333   
29996                    0  ...              2623    4648.333333   
29997                    3  ...              9155    3144.416667   
29998                    0  ...             47294    2298.250000   
29999                    0  ...             94268    8192.833333   

       UtilityBillsPaymentHistory  JobTenure  NetWorth  BaseInterestRate  \
0                        0.911812          9     58171          0.221832   
1                        0.926866          7     81104          0.241213   
2                        0.929530          5      1541          0.265370   
3                        0.953527          8      8247          0.210051   
4                        0.849103         10      9773          0.214471   
...                           ...        ...       ...               ...   
29995                    0.822208          8     28429          0.270762   
29996                    0.428198          3    100442          0.276013   
29997                    0.799783          1     51123          0.229399   
29998                    0.931287          6      1219          0.313287   
29999                    0.850134          4      9876          0.216469   

       InterestRate  MonthlyLoanPayment  TotalDebtToIncomeRatio  RiskScore  
0          0.223311          203.884443                0.259363       54.0  
1          0.227904          441.066918                0.318342       45.6  
2          0.298448         2293.941651                1.155993       51.0  
3          0.168041         1532.310234                0.523631       59.0  
4          0.221393          535.974330                0.096588       40.0  
...             ...                 ...                     ...        ...  
29995      0.246649         2300

In [22]:
X = df_test.drop(columns=['RiskScore'])
y = df_test['RiskScore']
X_scaled = scaler.transform(X)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.9, random_state=42)

## Testando com XGBoost

In [24]:
y_pred = xgboost_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred, squared=True)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse}\nRMSE: {rmse}\nMAE: {mae}\nMAPE: {mape}\nR2 Score: {r2}')

MSE: 5.615256427248078
RMSE: 2.36965322932451
MAE: 1.5214635333591036
MAPE: 0.032445969377887775
R2 Score: 0.9055376426105486


/opt/homebrew/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
